<a href="https://colab.research.google.com/github/mvdheram/DeepNLP/blob/master/DeepNLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural network 

Source : http://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes05-LM_RNN.pdf

**What is RNN**?

* CNN is pretty basic where the output is dependent on the input provided.
* When dealing with text sequences (time related dependence), the information from just the previous output might not be sufficient. Text sequences need information from past inputs.

* Language modelling task is a task similar where the problem is to predict the next word in a sequence based on previous sequence of words.
* In other words, language model is a task of assigning probabilty distribution to sequence of words.
  1. N-gram language model : Based on n-grams (n consecutive words occuring side by side) the probability of n-grams are calculated from the corpus. The probabilities are calculated based on the counting  and on markov assumuption that prob(n) depends on prob(n-1)). 
    * Problems:
      * **Sparsity problem** where the n-gram probability might be zero in some cases of sequences.
      * **Storing** n-gram probabilities.
  2. Window-based Neural Language Model : Neural Probabilistic Language Model. The model learns a distributed representation of each word along the window *(word embeddings - concatenated words/one-hot vectors)* and the probability function for *word sequences*. 
    * Problems:
      * **Fixed window** problem
      * Different weights used for different words which leads to **No symmetry** in how the inputs are processed.
* To handle these problems, RNN architecture has been used where the Recurrent Neural Networks (RNN) are capable of
conditioning the model on **all** previous words in the corpus.
* RNN architecture consists of recurrent unit (hidden layer / Memory unit) at each time step which holds a number of neurons and tells "which memories" to pay attention to when making new memory.
* This hidden layer performs a linear matrix operation followed by a non-linear operation (e.g. tanh())
* At each time-step, there are two inputs to hidden layer: 

    1.The output of the previous layer h(t-1)

    2.The input at that timestep x(t). 
    
    Since, time is the independent variable, the data variable need to be indexed by time. 

    ```x(t) - input; 
    h(t) - hidden unit dependent on input x(t) and h(t-1) output of the non-linear function at the previous time-step(t-1) ```
* The **output of previous hidden layer h(t-1)** is multiplied by a weight matrix **W(hh)**; while the **input x(t)** is multiplied by **w(hx)** to produce output features h(t).
* The **h(t)** produced is **multiplied** with weight matrix **W(s)** and run through softmax over vocab to obtain a prediction **output y(t)** of the next word. 

RNN Pipeline:

      Big picture : x(t) + h(t-1) -> h(t) -> y(t)

      Recurrent unit h(t) details : 
      h(t) = sigmoid_ActFunc [W(hh)*h(t-1) + W(hx)*x(t)]

      Output Probability distribution over vocab :
      y(t) = softmax[W(s)*h(t)]

      Shape(W(input)) = shape(x(t)) * shape(h(t))
      Shape(W(hidden)) = shape(h(t)) * shape(h(t))
      Shape(W(output)) = shape(h(t)) * shape(y(t))

* **Same weights W(hh) and W(hx) are applied repeatedly at each timestep.** Hence,
  1. The number of parameters
the model has to learn is less.
  2. The number of parameters is independent of the input sequence, thus defeating the curse of dimentionality!





**Why RNN architecture**?

* A feed forward neural network can represent/approximate any type of architecture.
*  The goal of a RNN implementation is to enable propagating context information through faraway time-steps.

RNN architecture vs feed forward neural network architecture when dealing with sequences.

```
Eg. 
T = 20 (sequence length)
D = 10 (input dimentionality/size)
M = 15 (hidden layer size)
K = 3 (number of output classes)
```

Feedforward:

```
Input-to-hidden weight matrix size = TxDxTxM = 60,000 weights in total 
Hidden-to-output weight matrix size = TxMxMxK = 18,000 weights in total 
Total : 78,000 parameters (Weights updates) 
```

RNN :

```
Input-to-hidden weight matrix size = DxM = 150 weights in total 
Hidden-to-output weiht matrix size = MxM = 225 weights in total
Hidden-to-output weight matrix size = MxK = 45 weights in total 
total = 420 parameters (Weights updates)
```
Hence, ANN lacks the following when dealing with sequences:

1. Due to parameters size.
2. Lack of structure. (as everything is connected to everything)

**RNN loss function and Perplexity of LM**:

Loss function is applied as sum over the entire vocabulary at time-step t. Perplexity of LM is the mearure of confusion and is the 2 to the power of negative log probability of the cross entropy error function.

Loss function : Cross entropy (-log loss)

perplexity : 2^cross entropy


**RNN advantages and disadvantages** :

Advantages:

1. Can process input sequences of any length.
2. Model size does not increase for longer input sequence
lengths.
3. The same weights are applied to every timestep of the input, so
there is symmetry in how inputs are processed.

Disadvantages:

1. Computation is slow - because it is sequential, it cannot be parallelized.
2. In practice, it is difficult to access information from many steps
back (long-term dependencies) due to problems like vanishing and exploding gradients.

Vanishing and explosion gradient problem :

* RNN propagates weight matrices from one-time step to another and when updating the weights during the backpropagation, the gradients values gradually vanishes as they propagate earlier in steps. Hence, for long sentences, the probability of predicting the correct next word decreases.

  * Two techniques to handle:
    1. Initialize the hidden weight as an identity matrix rather than random.
    2. Use Recitified Linear Units (ReLU) instead of sigmoid.

## Varients of RNN: GRU & LSTM

### Gated Recurrent Unit (GRU)

Why?

* To better handle long-term dependencies than above Simple Recurrent Unit (SRU).

How ?

* Introduces gates which allow the recurrent unit to remember or forget values.
* Using 4 componenets of GRU
  1. **New Memory Generation / Candidate state [˜h(t)]**: A new memory ˜h(t) is the consolidation of
a new input word x(t) with the past hidden state h(t−1) based on Reset gate.
  2. **Reset gate [r(t)]** : The reset signal r(t) is responsible for determining how
important h(t−1) is to the summarization ˜h(t). Values ranges between 0-1.
  3. **Update gate [z(t)]** : The update signal z(t) is responsible for determining how much of h(t−1) should be carried forward to the next state. Values ranges between 0-1. if z(t) ≈ 1, then past hidden state h(t−1) is almost entirely copied out to ht.
Conversely, if z(t) ≈ 0, then mostly the new memory ˜h(t)
is forwarded
to the next hidden state.
  4. **Hidden / next state [h(t)]** : The hidden state h(t)
is finally generated using the
past hidden input h(t−1) and the new memory generated ˜h(t) with the
advice of the update gate. 
  

[**Reset gate -> New memory** -> **update gate**] -> **hidden state / next state**   

Keras :

* To ouput the hidden state at a time step, pass in `return_state = Ture`. 

Eg.  output, h = GRU(128, return_state = True) (input)

**The recurrent layer output and hidden state output in a timestep are the same** 


In [13]:
# RNN test
from keras.models import Model
from keras.layers import Input, GRU
import numpy as np
import matplotlib.pyplot as plt


T = 8
D = 2
M = 3

X = np.random.randn(1,T,D)

def gru():
  input_ = Input(shape = (T,D))
  rnn = GRU(M, return_state = True)
  x = rnn(input_)

  model = Model(inputs = input_, outputs =x)
  o, h = model.predict(X)
  print("o:",o)
  print("h:",h)
  if(o==h).all():
    print("Output and hidden state are same")

def gru1():
  input_ = Input(shape = (T,D))
  rnn = GRU(M, return_state = True, return_sequences=True)
  x = rnn(input_)

  model = Model(inputs = input_, outputs =x)
  o, h = model.predict(X)
  print("o:",o)
  print("h:",h)
  if(o==h).all():
    print("Output and hidden state are same")

gru()
print("----------------")
gru1()
print("Length of output is 8 as length of sequences is 8")

o: [[0.3780474  0.07000626 0.29066744]]
h: [[0.3780474  0.07000626 0.29066744]]
Output and hidden state are same
----------------
o: [[[ 0.13738938  0.23651515 -0.00949176]
  [-0.19013974  0.07453187 -0.06851615]
  [-0.23519921 -0.0421285  -0.07376076]
  [-0.18998587  0.08858846 -0.10147636]
  [-0.2643824  -0.09200244 -0.09300735]
  [-0.00317092 -0.2182115   0.07165496]
  [-0.16735573 -0.02561122 -0.03028229]
  [-0.37837565  0.24546923 -0.12733148]]]
h: [[-0.37837565  0.24546923 -0.12733148]]
Length of o is 8 as return sequences return the last output


### Long-short Term Memory Unit (LSTM)

Why?

* To better handle long-term dependencies with more gates than GRU.

How ?

* Introduces more gates than GRU
  1. **New memory cell / candidate cell [c˜(t)]** : Compute new memory based on new input x(t) and hidden state h(t-1).  tanh activation used where values range between -1 to n .
  2. **Input gate [i(t)]** : Compute the importance of new input based on input x(t) and hidden state h(t-1). Sigmoid activation used where values range between 0-1.
  3. **Forget gate [f(t)]** : Computes the importance of past memory cell c(t-1) based on input x(t) and hidden state h(t-1). Sigmoid activation used where values range between 0-1. 
  4. **Final memory cell / cell state [c(t)]** :  Takes advice of forget gate f(t) which is based on past memory c(t-1) and input gate i(t) and new memory cell c˜(t) and sums up these two to produce the c(t).
  5. **Output/exposure gate / hidden state [h(t)]** : Computes how much of Final memory cell should be exposed.The signal it produces
to indicate this is o(t) and this is used to gate the point-wise tanh of
the memory c(t).

  **tanh[New memory] + sigmoid[input gate] + sigmoid[forget gate] -> cell state + hidden state**  

**Keras** :

* To ouput the hidden state at a time step, pass in `return_state = Ture`. 

Eg.  output, h, c = LSTM(128, return_state = True) (input)

* To output the sequence at each time step, pass in `return_sequences = True` or `return_sequences = False` to output the last timestep output.

Eg. output, h,c = LSTM(128, return_state = True, return_sequences = True) (input)

**The recurrent layer output and hidden state output in a timestep are the same**

In [15]:
# RNN test
from keras.models import Model
from keras.layers import Input, LSTM
import numpy as np
import matplotlib.pyplot as plt


T = 8
D = 2
M = 3

X = np.random.randn(1,T,D)

def lstm():
  input_ = Input(shape = (T,D))
  rnn = LSTM(M, return_state = True)
  x = rnn(input_)

  model = Model(inputs = input_, outputs =x)
  o, h, c = model.predict(X)
  print("o:",o)
  print("h:",h)
  print("c:",c)
  if(o==h).all():
    print("Output and hidden state are same")

def lstm1():
  input_ = Input(shape = (T,D))
  rnn = LSTM(M, return_state = True, return_sequences=True)
  x = rnn(input_)

  model = Model(inputs = input_, outputs =x)
  o, h, c = model.predict(X)
  print("o:",o)
  print("h:",h)
  print("c:",c)
  if(o==h).all():
    print("Output and hidden state are same")

lstm()
print("----------------")
lstm1()

o: [[-0.18850638  0.02607018 -0.08591584]]
h: [[-0.18850638  0.02607018 -0.08591584]]
c: [[-0.40824345  0.07652194 -0.26011327]]
Output and hidden state are same
----------------
o: [[[-0.12279311 -0.08188009  0.01587407]
  [-0.29582298  0.23334044 -0.3461163 ]
  [ 0.0368443   0.17599915 -0.3949932 ]
  [-0.09588622  0.40716726 -0.17474164]
  [-0.07354331  0.13536981 -0.11416193]
  [ 0.04567917 -0.01000865 -0.01194016]
  [ 0.13145879  0.03977356 -0.02190189]
  [ 0.09788015 -0.10855313  0.05099949]]]
h: [[ 0.09788015 -0.10855313  0.05099949]]
c: [[ 0.31439742 -0.23249981  0.16215065]]


### RNN tasks

RNN inputs:
  * sequence length x input dimentionality (TxD).

  Eg.

    Input sequence length (T) : "The quick brown fox jumps" : 5 words,
    input dimentionality (D): 50 dimentions/word, 

    Input_shape = 5 x 50

RNN outputs:
  * Sequence length x Number of classes (TxK)

  Eg. 

    Input sequence length (T) : "The quick brown fox jumps" : 5 words,
    Output Dimentionality (D) : 5 classes

    Output_shape = 5 x 5  

Types:

1. Many to one RNN: 
  * Take input sequence and produce a single output (Based on category or classes).
  * Eg. sentiment classification or spam classification
  * In Keras, 
    * Pass `input_sequence = false` to return only the last recurrent unit output.
    * Take a global_max_pool of all the recurrent unit which returns the maximum values out of all the recurrent units. 

      Eg. "Hi, I am nigerian prince ..." it is enought to look at the first few words, take gloabal max pool and predict output than taking the output based on the last word.  

2. Many to Many RNN:

  * Take input sequence of length T and produce T outputs. An output per time step.
  * Eg. Parts of speech POS-tagging, Named entity recognition. (Pass `input_sequence = true` to return only the last recurrent unit output.)
  * Eg. Machine translation, chatbots, question answering.
  * If length of input sequence is not equal to output sequence (Machine traslation English-Japanese)
    * Strategy : Pad sequences to equal length.
3. One to Many RNN:
  * Eg. Poetry generation. 


    
